# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

## Code

In [17]:
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [2]:
outage_df = pd.read_excel('outage.xlsx')
#CLEANING
a = dict(zip(list(pd.DataFrame(outage_df.iloc[5:, 1:]).columns), outage_df.iloc[4].tolist()[1:]))
outage_cleaned = pd.DataFrame(outage_df.iloc[6:, 1:]).rename(columns=a).reset_index().drop(['index'], axis='columns')
outage_cleaned['OUTAGE.START'] = (outage_cleaned['OUTAGE.START.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  outage_cleaned['OUTAGE.START.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
outage_cleaned['OUTAGE.RESTORATION'] = (outage_cleaned['OUTAGE.RESTORATION.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  outage_cleaned['OUTAGE.RESTORATION.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
outage_cleaned = outage_cleaned.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis='columns')

In [3]:
outage_cleaned

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION
0,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00
1,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,...,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00
2,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00
3,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00
4,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,public appeal,...,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00
1530,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,fuel supply emergency,...,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765,NaT,NaT
1531,1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,islanding,...,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00
1532,1533,2009,8,South Dakota,SD,MRO,West North Central,0.5,warm,islanding,...,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256,2009-08-29 11:00:00,2009-08-29 14:01:00


In [147]:
from sklearn.impute import SimpleImputer, KNNImputer

In [71]:
# woah = outage_cleaned[outage_cleaned['CLIMATE.CATEGORY'].notna() & outage_cleaned['TOTAL.PRICE'].notna()]
woah = outage_cleaned.copy()
woah = woah.dropna(subset=['CLIMATE.CATEGORY', 'TOTAL.PRICE', 'POPPCT_URBAN', 'OUTAGE.DURATION'])
woah['MONTH'] = woah['MONTH'].transform(str)

In [124]:
prepoc = ColumnTransformer([
    # ('imputation', SimpleImputer(strategy='median'), ['CUSTOMERS.AFFECTED']),
    ('climate', OneHotEncoder(handle_unknown='ignore'), ['CLIMATE.CATEGORY']),
    ('test', FunctionTransformer(lambda x: x), ['TOTAL.PRICE']),
], remainder='drop')

imputer = ColumnTransformer([
    ('imputation', SimpleImputer(strategy='median'), ['CUSTOMERS.AFFECTED']),
], remainder='passthrough')

pl = Pipeline([
    ('impute', imputer),
    ('prepoc', prepoc),
    ('cook', RandomForestClassifier())
])

X_train, X_test, y_train, y_test = train_test_split(woah.drop('MONTH', axis=1), woah['MONTH'], test_size=0.2)

pl.fit(X_train, y_train)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [121]:
pl.score(X_test, y_test)

0.3003412969283277

In [100]:
from sklearn.model_selection import GridSearchCV

params = {
    'cook__n_estimators': [50, 75, 100, 125, 150, 200, 250, 300], 
    'cook__max_depth': [1, 3, 5, 7, 9, 12, 15, 18, 20], 
    'cook__min_samples_split': [2, 5, 8, 10, 12, 15]
}

grid_search = GridSearchCV(estimator=pl, param_grid=params, cv=5)
grid_search.fit(X_train, y_train)

/Users/walterwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1680 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1680 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/walterwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/walterwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/walterwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklear

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prepoc',
                                        ColumnTransformer(transformers=[('climate',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['CLIMATE.CATEGORY']),
                                                                        ('test',
                                                                         FunctionTransformer(func=<function <lambda> at 0x2916c5550>),
                                                                         ['TOTAL.PRICE'])])),
                                       ('cook', RandomForestClassifier())]),
             param_grid={'cook__max_features': [1, 3, 5, 7, 9, 12, 15, 18, 20],
                         'cook__min_samples_split': [2, 5, 8, 10, 12, 15],
                         'cook__n_estimators': [50, 75, 100, 125, 150, 200, 250,
               

In [101]:
grid_search.best_params_

{'cook__max_features': 1,
 'cook__min_samples_split': 2,
 'cook__n_estimators': 50}

In [123]:
pl.score(X_train, y_train)

0.9829205807002562

In [95]:
pl.score(X_test, y_test)

0.49829351535836175

In [14]:
woah['TOTAL.PRICE'].isna().sum()

13

In [145]:
# woah = outage_cleaned[outage_cleaned['CLIMATE.CATEGORY'].notna() & outage_cleaned['TOTAL.PRICE'].notna()]
woah = outage_cleaned.copy()
woah = woah.dropna(subset=['CLIMATE.CATEGORY', 'TOTAL.PRICE', 'POPPCT_URBAN', 'OUTAGE.DURATION', 'RES.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS'])
woah['MONTH'] = woah['MONTH'].transform(str)
# woah['CUSTOMERS.AFFECTED'] = woah['CUSTOMERS.AFFECTED'] + 1

In [148]:
column_to_impute = 'CUSTOMERS.AFFECTED'

# Create a KNNImputer object
imputer = KNNImputer(n_neighbors=5)

# Impute missing values in the specified column
imputed_data = pd.DataFrame(imputer.fit_transform(woah[[column_to_impute]]))

# Update the original DataFrame with the imputed values
woah.loc[:, column_to_impute] = imputed_data.values.ravel()

In [196]:
prepoc = ColumnTransformer([
    # ('imputation', SimpleImputer(strategy='median'), ['CUSTOMERS.AFFECTED']),
    ('climate', OneHotEncoder(handle_unknown='ignore'), ['CLIMATE.CATEGORY']),
    # ('log', FunctionTransformer(lambda x: np.log(x + 1)), ['CUSTOMERS.AFFECTED']),
    ('test', FunctionTransformer(lambda x: x), ['TOTAL.PRICE', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'RES.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS']),
], remainder='drop')

# imputer = ColumnTransformer([
#     ('imputation', SimpleImputer(strategy='median'), ['CUSTOMERS.AFFECTED']),
# ], remainder='passthrough')

pl = Pipeline([
    # ('impute', imputer),
    ('prepoc', prepoc),
    ('cook', RandomForestClassifier(max_depth=10))
])

X_train, X_test, y_train, y_test = train_test_split(woah.drop('U.S._STATE', axis=1), woah['U.S._STATE'], test_size=0.2)

pl.fit(X_train, y_train)

Pipeline(steps=[('prepoc',
                 ColumnTransformer(transformers=[('climate',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['CLIMATE.CATEGORY']),
                                                 ('test',
                                                  FunctionTransformer(func=<function <lambda> at 0x2916db160>),
                                                  ['TOTAL.PRICE',
                                                   'CUSTOMERS.AFFECTED',
                                                   'OUTAGE.DURATION',
                                                   'RES.PRICE', 'TOTAL.SALES',
                                                   'TOTAL.CUSTOMERS'])])),
                ('cook', RandomForestClassifier(max_depth=10))])

In [197]:
pl.score(X_test, y_test)

0.8873720136518771

In [198]:
pl.score(X_train, y_train)

0.9888983774551665

In [272]:
# woah = outage_cleaned[outage_cleaned['CLIMATE.CATEGORY'].notna() & outage_cleaned['TOTAL.PRICE'].notna()]
woah = outage_cleaned.copy()
woah = woah.dropna(subset=['CLIMATE.CATEGORY', 'TOTAL.PRICE', 'POPPCT_URBAN', 'OUTAGE.DURATION', 'RES.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS'])
woah['MONTH'] = woah['MONTH'].transform(str)
# woah['CUSTOMERS.AFFECTED'] = woah['CUSTOMERS.AFFECTED'] + 1
column_to_impute = 'CUSTOMERS.AFFECTED'

# Create a KNNImputer object
imputer = KNNImputer(n_neighbors=5)

# Impute missing values in the specified column
imputed_data = pd.DataFrame(imputer.fit_transform(woah[[column_to_impute]]))

# Update the original DataFrame with the imputed values
woah.loc[:, column_to_impute] = imputed_data.values.ravel()

prepoc = ColumnTransformer([
    # ('imputation', SimpleImputer(strategy='median'), ['CUSTOMERS.AFFECTED']),
    ('climate', OneHotEncoder(handle_unknown='ignore'), ['CLIMATE.CATEGORY']),
    ('log', FunctionTransformer(lambda x: np.log(x + 1)), ['CUSTOMERS.AFFECTED']),
    ('test', FunctionTransformer(lambda x: x), ['TOTAL.PRICE', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'RES.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS']),
], remainder='drop')

# imputer = ColumnTransformer([
#     ('imputation', SimpleImputer(strategy='median'), ['CUSTOMERS.AFFECTED']),
# ], remainder='passthrough')

pl = Pipeline([
    # ('impute', imputer),
    ('prepoc', prepoc),
    ('cook', RandomForestClassifier(max_depth=20, min_samples_split=2, n_estimators=125))
    # ('cook', RandomForestClassifier())
])

X_train, X_test, y_train, y_test = train_test_split(woah.drop('CAUSE.CATEGORY', axis=1), woah['CAUSE.CATEGORY'], test_size=0.2)

pl.fit(X_train, y_train)

Pipeline(steps=[('prepoc',
                 ColumnTransformer(transformers=[('climate',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['CLIMATE.CATEGORY']),
                                                 ('log',
                                                  FunctionTransformer(func=<function <lambda> at 0x29af6f790>),
                                                  ['CUSTOMERS.AFFECTED']),
                                                 ('test',
                                                  FunctionTransformer(func=<function <lambda> at 0x2913a7c10>),
                                                  ['TOTAL.PRICE',
                                                   'CUSTOMERS.AFFECTED',
                                                   'OUTAGE.DURATION',
                                                   'RES.PRICE', 'TOTAL.SALES',
                                               

In [273]:
pl.score(X_test, y_test)

0.8327645051194539

In [274]:
pl.score(X_train, y_train)

1.0

In [261]:
from sklearn.model_selection import GridSearchCV

params = {
    'cook__n_estimators': [100, 125, 150, 200, 250], 
    'cook__max_depth': [15, 18, 20, 22, 25, 30, 40], 
    'cook__min_samples_split': [2, 5, 8]
}

grid_search = GridSearchCV(estimator=pl, param_grid=params, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prepoc',
                                        ColumnTransformer(transformers=[('climate',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['CLIMATE.CATEGORY']),
                                                                        ('log',
                                                                         FunctionTransformer(func=<function <lambda> at 0x29071eca0>),
                                                                         ['CUSTOMERS.AFFECTED']),
                                                                        ('test',
                                                                         FunctionTransformer(func=<function <lambda> at 0x29108e040>),
                                                                         ['TOTAL.PRICE',
                    

In [262]:
grid_search.best_params_

{'cook__max_depth': 20,
 'cook__min_samples_split': 2,
 'cook__n_estimators': 150}

### Framing the Problem

### Baseline Model

In [ ]:
# TODO

### Final Model

In [ ]:
# TODO

### Fairness Analysis